## Loading the Model

In [1]:
import pickle

# Enter Path of model 
filename = 'finalized_model.sav'

loaded_model = pickle.load(open(filename, 'rb'))

In [2]:
import pandas as pd
import numpy as np
from scipy.stats import mode

## Loading Evaluation Dataset

### Load evaluation sales dataset here : Daily dataset required - monthly aggreating function is described later

In [3]:
# Enter Path of sales data
path_sales = ''
eval_data_sales = pd.read_csv(path_sales,parse_dates=['date'])

In [5]:
# Enter Path of product_hierarchy data
path_product = ''
eval_data_product_hierarchy = pd.read_csv(path_product)

In [6]:
# Enter Path of store_cities data
path_cities = ''
eval_data_store_cities = pd.read_csv(path_cities)

# Filling Nas for Product

In [7]:
eval_data_product_hierarchy['product_length'].fillna(eval_data_product_hierarchy['product_length'].mean(),inplace = True)
eval_data_product_hierarchy['product_depth'].fillna(eval_data_product_hierarchy['product_depth'].mean(),inplace = True)
eval_data_product_hierarchy['product_width'].fillna(eval_data_product_hierarchy['product_width'].mean(),inplace = True)
eval_data_product_hierarchy['cluster_id'].fillna(mode(eval_data_product_hierarchy['cluster_id'])[0][0],inplace = True)

In [8]:
eval_data_product_hierarchy.isna().sum()

product_id        0
product_length    0
product_depth     0
product_width     0
cluster_id        0
hierarchy1_id     0
hierarchy2_id     0
hierarchy3_id     0
hierarchy4_id     0
hierarchy5_id     0
dtype: int64

# Filling Nas for Sales dataset

In [9]:
eval_data_sales['promo_bin_1'].fillna(0,inplace= True)
eval_data_sales['promo_bin_2'].fillna(0,inplace= True)
eval_data_sales['promo_discount_2'].fillna(0,inplace= True)
eval_data_sales['promo_discount_type_2'].fillna(0,inplace= True)

eval_data_sales['revenue'].fillna(eval_data_sales['revenue'].mean(),inplace = True)
eval_data_sales['stock'].fillna(eval_data_sales['stock'].mean(),inplace = True)

In [10]:
dict_price = {}
for i, x in enumerate(eval_data_sales['product_id']):
    if (x not in dict_price.keys() and not(np.isnan(eval_data_sales['price'][i]))):
        dict_price[x]=eval_data_sales['price'][i]

eval_data_sales.price = eval_data_sales.price.fillna(eval_data_sales.product_id.map(dict_price))

eval_data_sales['price'].fillna(eval_data_sales['price'].mean(),inplace = True)

## Categorical to Numerical for promos

In [11]:
encode_dict_type_1 = {
    'PR01':1,
    'PR03':3,
    'PR04':4,
    'PR05':5,
    'PR06':6,
    'PR07':7,
    'PR08':8,
    'PR09':9,
    'PR10':10,
    'PR11':11,
    'PR12':12,
    'PR13':13,
    'PR14':14,
    'PR15':15,
    'PR16':16,
    'PR17':17,
    'PR18':18
}

eval_data_sales['promo_type_1'].replace(encode_dict_type_1,inplace=True)

In [12]:
encode_dict = {
    'verylow':1,
    'low':2,
    'moderate':3,
    'high':4,
    'veryhigh':5
}

eval_data_sales['promo_bin_1'].replace(encode_dict,inplace=True)

In [13]:
encode_dict_type_2 = {'PR01':1,
                     'PR02':2,
                     'PR03':3,
                     'PR04':4}

eval_data_sales['promo_type_2'].replace(encode_dict_type_2,inplace=True)

In [14]:
encode_dict_bin_2 = {
    'verylow':1,
    'low':2,
    'high':3,
    'veryhigh':4
}


eval_data_sales['promo_bin_2'].replace(encode_dict_bin_2,inplace=True)

In [15]:
encode_discount_type_2 = {'PR01':1,
                     'PR02':2,
                     'PR03':3,
                     'PR04':4}

eval_data_sales['promo_discount_type_2'].replace(encode_discount_type_2,inplace=True)

# Converting to weekly data

In [16]:

eval_data_sales = eval_data_sales.groupby(['product_id','store_id']).resample('W', label='right', closed = 'right', on='date').sum().reset_index()
eval_data_sales

,product_id,store_id,date,revenue,stock,price,promo_type_1,promo_bin_1,promo_type_2,promo_bin_2,promo_discount_2,promo_discount_type_2
0,P0001,S0001,2017-02-12,11.02,0.0,13.00,28,0,6,0.0,0.0,0.0
1,P0001,S0001,2017-02-19,0.00,0.0,0.00,0,0,0,0.0,0.0,0.0
2,P0001,S0001,2017-02-26,5.51,0.0,6.50,14,0,3,0.0,0.0,0.0
3,P0001,S0002,2017-01-08,10.59,52.0,43.75,98,0,21,0.0,0.0,0.0
4,P0001,S0002,2017-01-15,16.32,32.0,44.25,98,0,21,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
154591,P0747,S0144,2017-02-19,0.00,7.0,92.75,98,0,21,0.0,0.0,0.0
154592,P0747,S0144,2017-02-26,0.00,7.0,92.75,98,0,21,0.0,0.0,0.0
154593,P0747,S0144,2017-03-05,0.00,7.0,92.75,98,0,21,0.0,0.0,0.0
154594,P0747,S0144,2017-03-12,0.00,23.0,92.75,98,0,21,0.0,0.0,0.0


In [42]:
## Creating Final Predicted Dataset

final_predicted_dataset = pd.DataFrame()
final_predicted_dataset['product_id'] = eval_data_sales['product_id']
final_predicted_dataset['store_id'] = eval_data_sales['store_id']
final_predicted_dataset['date'] = eval_data_sales['date']

In [17]:
## Filling correct prices for sales

eval_data_sales = eval_data_sales.drop(['price'],axis = 1)

price = []
for x in eval_data_sales['product_id']:
    if x in dict_price.keys():
        price.append(dict_price[x])
    else :
        price.append(np.nan)

eval_data_sales['price'] = price
eval_data_sales['price'].fillna(method = 'ffill',inplace = True)

In [18]:
#Combining product_hierarchy features with sales

dict_length = dict(zip(eval_data_product_hierarchy.product_id,eval_data_product_hierarchy.product_length))
dict_depth = dict(zip(eval_data_product_hierarchy.product_id,eval_data_product_hierarchy.product_depth))
dict_width = dict(zip(eval_data_product_hierarchy.product_id,eval_data_product_hierarchy.product_width))
dict_cluster_id = dict(zip(eval_data_product_hierarchy.product_id,eval_data_product_hierarchy.cluster_id))
dict_hierarchy1_id = dict(zip(eval_data_product_hierarchy.product_id,eval_data_product_hierarchy.hierarchy1_id))
dict_hierarchy2_id = dict(zip(eval_data_product_hierarchy.product_id,eval_data_product_hierarchy.hierarchy2_id))
dict_hierarchy3_id = dict(zip(eval_data_product_hierarchy.product_id,eval_data_product_hierarchy.hierarchy3_id))
dict_hierarchy4_id = dict(zip(eval_data_product_hierarchy.product_id,eval_data_product_hierarchy.hierarchy4_id))
dict_hierarchy5_id = dict(zip(eval_data_product_hierarchy.product_id,eval_data_product_hierarchy.hierarchy5_id))



product_length = []
product_depth = []
product_width = []
cluster_id = []
hierarchy1_id = []
hierarchy2_id = []
hierarchy3_id = []
hierarchy4_id = []
hierarchy5_id = []

for x in eval_data_sales['product_id']:
    product_length.append(dict_length[x])
    product_depth.append(dict_depth[x])
    product_width.append(dict_width[x])
    cluster_id.append(dict_cluster_id[x])
    hierarchy1_id.append(dict_hierarchy1_id[x])
    hierarchy2_id.append(dict_hierarchy2_id[x])
    hierarchy3_id.append(dict_hierarchy3_id[x])
    hierarchy4_id.append(dict_hierarchy4_id[x])
    hierarchy5_id.append(dict_hierarchy5_id[x])

eval_data_sales['product_length'] = product_length
eval_data_sales['product_depth'] = product_depth
eval_data_sales['product_width'] = product_width
eval_data_sales['cluster_id'] = cluster_id
eval_data_sales['hierarchy1_id'] = hierarchy1_id
eval_data_sales['hierarchy2_id'] = hierarchy2_id
eval_data_sales['hierarchy3_id'] = hierarchy3_id
eval_data_sales['hierarchy4_id'] = hierarchy4_id
eval_data_sales['hierarchy5_id'] = hierarchy5_id

In [19]:
#Combining store_cities features with sales

dict_storetype_id = dict(zip(eval_data_store_cities.store_id,eval_data_store_cities.storetype_id))
dict_store_size = dict(zip(eval_data_store_cities.store_id,eval_data_store_cities.store_size))
dict_city_id = dict(zip(eval_data_store_cities.store_id,eval_data_store_cities.city_id))

storetype_id = []
store_size = []
city_id = []

for x in eval_data_sales['store_id']:
    storetype_id.append(dict_storetype_id[x])
    store_size.append(dict_store_size[x])
    city_id.append(dict_city_id[x])

eval_data_sales['storetype_id'] = storetype_id
eval_data_sales['store_size'] = store_size
eval_data_sales['city_id'] = city_id  

In [20]:
# Encode Remaining Labels

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
eval_data_sales['product_id'] = le.fit_transform(eval_data_sales['product_id'])
eval_data_sales['store_id'] = le.fit_transform(eval_data_sales['store_id'])
eval_data_sales['cluster_id'] = le.fit_transform(eval_data_sales['cluster_id'])
eval_data_sales['hierarchy1_id'] = le.fit_transform(eval_data_sales['hierarchy1_id'])
eval_data_sales['hierarchy2_id'] = le.fit_transform(eval_data_sales['hierarchy2_id'])
eval_data_sales['hierarchy3_id'] = le.fit_transform(eval_data_sales['hierarchy3_id'])
eval_data_sales['hierarchy4_id'] = le.fit_transform(eval_data_sales['hierarchy4_id'])
eval_data_sales['hierarchy5_id'] = le.fit_transform(eval_data_sales['hierarchy5_id'])
eval_data_sales['storetype_id'] = le.fit_transform(eval_data_sales['storetype_id'])
eval_data_sales['store_size'] = le.fit_transform(eval_data_sales['store_size'])
eval_data_sales['city_id'] = le.fit_transform(eval_data_sales['city_id'])

# Final Testing dataset that will go to model

In [41]:
Final_X_test = pd.DataFrame()

In [32]:
Final_X_test['product_id'] = eval_data_sales['product_id']
Final_X_test['store_id'] = eval_data_sales['store_id']
Final_X_test['revenue'] = eval_data_sales['revenue']
Final_X_test['stock'] = eval_data_sales['stock']
Final_X_test['promo_type_1'] = eval_data_sales['promo_type_1']
Final_X_test['promo_type_2'] = eval_data_sales['promo_type_2']
Final_X_test['price'] = eval_data_sales['price']
Final_X_test['product_length'] = eval_data_sales['product_length']
Final_X_test['product_depth'] = eval_data_sales['product_depth']
Final_X_test['product_width'] = eval_data_sales['product_width']
Final_X_test['cluster_id'] = eval_data_sales['cluster_id']
Final_X_test['hierarchy1_id'] = eval_data_sales['hierarchy1_id']
Final_X_test['hierarchy2_id'] = eval_data_sales['hierarchy2_id']
Final_X_test['hierarchy3_id'] = eval_data_sales['hierarchy3_id']
Final_X_test['hierarchy4_id'] = eval_data_sales['hierarchy4_id']
Final_X_test['hierarchy5_id'] = eval_data_sales['hierarchy5_id']
Final_X_test['storetype_id'] = eval_data_sales['storetype_id']
Final_X_test['store_size'] = eval_data_sales['store_size']
Final_X_test['city_id'] = eval_data_sales['city_id']

In [33]:
Final_X_test

,product_id,store_id,revenue,stock,promo_type_1,promo_type_2,price,product_length,product_depth,product_width,cluster_id,hierarchy1_id,hierarchy2_id,hierarchy3_id,hierarchy4_id,hierarchy5_id,storetype_id,store_size,city_id
0,0,0,11.02,0.0,28,6,6.25,13.5,22.0,20.0,5,1,5,24,48,81,3,30,27
1,0,0,0.00,0.0,0,0,6.25,13.5,22.0,20.0,5,1,5,24,48,81,3,30,27
2,0,0,5.51,0.0,14,3,6.25,13.5,22.0,20.0,5,1,5,24,48,81,3,30,27
3,0,1,10.59,52.0,98,21,6.25,13.5,22.0,20.0,5,1,5,24,48,81,3,28,5
4,0,1,16.32,32.0,98,21,6.25,13.5,22.0,20.0,5,1,5,24,48,81,3,28,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
154591,335,116,0.00,7.0,98,21,11.95,23.0,23.0,33.3,0,1,7,29,60,102,3,12,20
154592,335,116,0.00,7.0,98,21,11.95,23.0,23.0,33.3,0,1,7,29,60,102,3,12,20
154593,335,116,0.00,7.0,98,21,11.95,23.0,23.0,33.3,0,1,7,29,60,102,3,12,20
154594,335,116,0.00,23.0,98,21,11.95,23.0,23.0,33.3,0,1,7,29,60,102,3,12,20


## Weekly Predictions 

In [39]:
y_pred = loaded_model.predict(Final_X_test)

In [43]:
final_predicted_dataset['y_pred'] = y_pred

In [ ]:
final_predicted_dataset